In [2]:
import numpy as np
from sklearn.pipeline import Pipeline

import GEOparse
from Genobolitics import *
from sklearn_utils.preprocessing import FoldChangeScaler
from metabolitics.preprocessing import MetaboliticsTransformer, MetaboliticsPipeline

import pickle

In [3]:
import sys, cplex

sys.path

['',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python35.zip',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/lib-dynload',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/site-packages',
 '/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/site-packages/IPython/extensions',
 '/home/abdu/.ipython']

In [5]:
breast_cancer_dataset = GEOparse.get_GEO('GDS5819').table

05-Jan-2018 11:51:12 INFO GEOparse - File already exist: using local version.
05-Jan-2018 11:51:12 INFO GEOparse - Parsing ./GDS5819.soft.gz: 
05-Jan-2018 11:51:12 DEBUG GEOparse - DATABASE: Geo
05-Jan-2018 11:51:12 DEBUG GEOparse - DATASET: GDS5819
05-Jan-2018 11:51:12 DEBUG GEOparse - SUBSET: GDS5819_1
05-Jan-2018 11:51:12 DEBUG GEOparse - SUBSET: GDS5819_2
05-Jan-2018 11:51:12 DEBUG GEOparse - SUBSET: GDS5819_3
05-Jan-2018 11:51:12 DEBUG GEOparse - SUBSET: GDS5819_4
05-Jan-2018 11:51:12 DEBUG GEOparse - DATASET: GDS5819
/home/abdu/Desktop/virtualenv/GERA/lib/python3.5/site-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [6]:
unhealthy_raw='\
#GSM1599177 = Value for GSM1599177: Primary human monocytes from patient with metastatic breast cancer #1; src: Primary human monocytes\
#GSM1599178 = Value for GSM1599178: Primary human monocytes from patient with metastatic breast cancer #2; src: Primary human monocytes\
#GSM1599179 = Value for GSM1599179: Primary human monocytes from patient with metastatic breast cancer #3; src: Primary human monocytes\
#GSM1599180 = Value for GSM1599180: Primary human monocytes from patient with metastatic breast cancer #4; src: Primary human monocytes'

In [7]:
healthy_raw='\
#GSM1599181 = Value for GSM1599181: Primary human monocytes from healthy control #1; src: Primary human monocytes\
#GSM1599182 = Value for GSM1599182: Primary human monocytes from healthy control #2; src: Primary human monocytes\
#GSM1599183 = Value for GSM1599183: Primary human monocytes from healthy control #3; src: Primary human monocytes'

In [8]:
clean = lambda string: string.replace(':', '').split()
predicate = lambda word: word.startswith('GSM')

healthy = set(filter(predicate, clean(healthy_raw)))
unhealthy = set(filter(predicate, clean(unhealthy_raw)))

In [9]:
print(len(healthy), len(unhealthy))

3 4


In [10]:
healthy & unhealthy

set()

In [8]:
breast_cancer_dataset[breast_cancer_dataset.columns[:5]].head()

,ID_REF,IDENTIFIER,GSM1599177,GSM1599178,GSM1599179
0,ILMN_1343048,ILMN_1343048,NaN,NaN,NaN
1,ILMN_1343049,ILMN_1343049,NaN,NaN,NaN
2,ILMN_1343050,ILMN_1343050,NaN,NaN,NaN
3,ILMN_1343052,ILMN_1343052,NaN,NaN,NaN
4,ILMN_1343059,ILMN_1343059,NaN,NaN,NaN


In [9]:
labels_ = [(h, 'healthy') for h in healthy] + [(d, 'unhealthy') for d in unhealthy]

In [10]:
X, y = parse_database('GDS5819', labels=dict(labels_))

04-Jan-2018 23:50:32 INFO GEOparse - File already exist: using local version.
04-Jan-2018 23:50:32 INFO GEOparse - Parsing ./GDS5819.soft.gz: 
04-Jan-2018 23:50:32 DEBUG GEOparse - DATABASE: Geo
04-Jan-2018 23:50:32 DEBUG GEOparse - DATASET: GDS5819
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_1
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_2
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_3
04-Jan-2018 23:50:32 DEBUG GEOparse - SUBSET: GDS5819_4
04-Jan-2018 23:50:32 DEBUG GEOparse - DATASET: GDS5819
/usr/local/lib/python3.5/dist-packages/GEOparse/GEOparse.py:330 FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


GSM1599180 added with length 9745
GSM1599182 added with length 9745
GSM1599177 added with length 9745
GSM1599179 added with length 9745
GSM1599178 added with length 9745
GSM1599183 added with length 9745
GSM1599181 added with length 9745


In [11]:
pickle.dump(X, open("datasets/breast_cancer_X","wb"))
pickle.dump(y, open("datasets/breast_cancer_y","wb"))

In [5]:
X = pickle.load(open("datasets/breast_cancer_X","rb"))
y = pickle.load(open("datasets/breast_cancer_y","rb"))

In [ ]:
genobolitics_transformer = MetaboliticsTransformer()
genobolitics_transformer.analyzer = Genobolitics("recon2")

pipeline = Pipeline([('scaling', FoldChangeScaler(reference_label='healthy')),
                     ('LP_FVA', genobolitics_transformer)])

results = pipeline.fit_transform(X, y)

pickle.dump(results, open("breast_cancer.results", "wb"))

/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
/home/abdu/Desktop/GERA/Genobolitics.py:36 UserWarning: some reaction genes fold changes were not found, reaction is discarded!
